In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import copy
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
cpu = torch.device('cpu')#"cuda:0" if torch.cuda.is_available() else "cpu")
dv=torch.device('cuda')#torch.device("cuda" if torch.cuda.is_available() else "cpu")import torchmetrics
import torchmetrics
acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)

In [2]:
from load_celebA import load_celeba

In [ ]:
import torchvision.models as pretrain
model_pre=pretrain.resnet18(pretrained=True)

In [4]:
for par in model_pre.parameters():
    par.requires_grad=False


In [5]:
from model_class import STL_vission
from my_util import DM_rate, branches,fair_loss,find_groups

In [6]:
dt,yt, gt, d_v, yv, gv, dataloader_test, y_test, g_test,batch_size=load_celeba(protected='Age')
N_tasks=len(y_test)
#change protected='Gender' for CelebA-Gender

In [7]:
shapes=[1024,128,256,512,128,64,256,128]
depth=len(shapes)-1

SLs=[nn.DataParallel(STL_vission(pretrain=model_pre,shapes=shapes)).to(dv) for t in range(N_tasks)]
SL_optis=[optim.AdamW(SLs[t].parameters()) for t in range(N_tasks)]
spaths={'path'+str(t):'/yourpath/Grouping_FMTL/models/stl/age_stl'+str(t)+'.pt' for t in range(N_tasks)}

In [8]:
criteria = nn.CrossEntropyLoss()
#best_S=[[0,1] for t in range(N_tasks)]

for epoch in range(50):  # loop over each NN multiple times
    loss_pointer=[]
    state_net,state_a,state_f=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    R_A,R_F=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    ch=np.random.choice([0,1,2,3])
    dataloader,y_train,g_train=dt[ch],yt[ch],gt[ch]
    y_train=[torch.tensor(y_train[i]) for i in range(N_tasks)]
    xg=torch.tensor(g_train)
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs =data[0]
        if ((i+1)*batch_size)<len(g_train):            
            labels=[y_train[t][i*batch_size:(i+1)*batch_size] for t in range(N_tasks)]#,y3_train[i:i+batch]]
            #if epoch<pretrn:
            xc=xg[i*batch_size:(i+1)*batch_size]
        else:
            labels=[y_train[t][i*batch_size:] for t in range(N_tasks)]#,y2_train[i:]]#,y3_train[i:]]
            #if epoch<pretrn:
            xc=xg[i*batch_size:]
        
        # zero the parameter gradients
        for t in range(N_tasks):
            ##training STLs
            SL_optis[t].zero_grad()
            out=SLs[t](inputs.to(dv).float())
            loss_Sa=criteria(out, labels[t].to(dv))
            loss_Sf=fair_loss(out, labels[t].to(dv),xc.to(dv))
            loss=loss_Sa+loss_Sf
            loss.backward()
            SL_optis[t].step()
            
            
            
            
    with torch.no_grad():
        ch=np.random.choice([0,1])
        val_batch,y_v,g_val=next(iter(d_v[ch]))[0],yv[ch],gv[ch]
       
        for t in range(N_tasks): 
            pred_SL=SLs[t](val_batch.to(dv).float())
            SL_acc=acc(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu))
            SL_eo=DM_rate(pred_SL.to(cpu),torch.tensor(y_v[t]).to(cpu),torch.tensor(g_val).to(cpu))
            torch.save(SLs[t].state_dict(),spaths['path'+str(t)])
            print('Task',t,'  Acc:',SL_acc, 'EO:',SL_eo)         

Task 0   Acc: tensor(0.8833) EO: (tensor(0.0134), tensor(0.0039))
Task 1   Acc: tensor(0.7425) EO: (tensor(0.0082), tensor(0.0072))
Task 2   Acc: tensor(0.6524) EO: (tensor(0.0376), tensor(0.0117))
Task 3   Acc: tensor(0.7890) EO: (tensor(0.0090), tensor(0.0065))
Task 4   Acc: tensor(0.8506) EO: (tensor(0.0202), tensor(0.0136))
Task 5   Acc: tensor(0.8484) EO: (tensor(0.0343), tensor(0.0314))
Task 6   Acc: tensor(0.7483) EO: (tensor(0.0201), tensor(0.0175))
Task 7   Acc: tensor(0.7930) EO: (tensor(0.0265), tensor(0.0154))
Task 8   Acc: tensor(0.8496) EO: (tensor(0.0115), tensor(0.0105))
Task 9   Acc: tensor(0.7575) EO: (tensor(0.0398), tensor(0.0259))
Task 10   Acc: tensor(0.8591) EO: (tensor(0.0118), tensor(0.0020))
Task 11   Acc: tensor(0.9296) EO: (tensor(0.0216), tensor(0.0134))
Task 12   Acc: tensor(0.9251) EO: (tensor(0.0047), tensor(0.0021))
Task 13   Acc: tensor(0.7481) EO: (tensor(0.0024), tensor(0.0009))
Task 14   Acc: tensor(0.6255) EO: (tensor(0.0047), tensor(0.0036))
Task 

Task 30   Acc: tensor(0.9273) EO: (tensor(0.0204), tensor(0.0171))
Task 0   Acc: tensor(0.8833) EO: (tensor(0.0134), tensor(0.0039))
Task 1   Acc: tensor(0.7615) EO: (tensor(0.0045), tensor(0.0013))
Task 2   Acc: tensor(0.6947) EO: (tensor(0.0256), tensor(0.0043))
Task 3   Acc: tensor(0.7890) EO: (tensor(0.0090), tensor(0.0065))
Task 4   Acc: tensor(0.8694) EO: (tensor(0.0153), tensor(0.0034))
Task 5   Acc: tensor(0.8502) EO: (tensor(0.0488), tensor(0.0458))
Task 6   Acc: tensor(0.7628) EO: (tensor(0.0271), tensor(0.0208))
Task 7   Acc: tensor(0.8328) EO: (tensor(0.0179), tensor(0.0155))
Task 8   Acc: tensor(0.9085) EO: (tensor(0.0673), tensor(0.0660))
Task 9   Acc: tensor(0.7738) EO: (tensor(0.0090), tensor(0.0024))
Task 10   Acc: tensor(0.8591) EO: (tensor(0.0118), tensor(0.0020))
Task 11   Acc: tensor(0.9296) EO: (tensor(0.0216), tensor(0.0134))
Task 12   Acc: tensor(0.9251) EO: (tensor(0.0047), tensor(0.0021))
Task 13   Acc: tensor(0.7900) EO: (tensor(0.0026), tensor(0.0024))
Task 

Task 29   Acc: tensor(0.8790) EO: (tensor(0.0330), tensor(0.0311))
Task 30   Acc: tensor(0.9295) EO: (tensor(0.0163), tensor(0.0161))
Task 0   Acc: tensor(0.8806) EO: (tensor(0.0076), tensor(0.0067))
Task 1   Acc: tensor(0.7645) EO: (tensor(0.0343), tensor(0.0204))
Task 2   Acc: tensor(0.7077) EO: (tensor(0.0220), tensor(0.0095))
Task 3   Acc: tensor(0.7962) EO: (tensor(0.0126), tensor(0.0011))
Task 4   Acc: tensor(0.8790) EO: (tensor(0.0433), tensor(0.0387))
Task 5   Acc: tensor(0.8473) EO: (tensor(0.0107), tensor(0.0066))
Task 6   Acc: tensor(0.7711) EO: (tensor(0.0186), tensor(0.0155))
Task 7   Acc: tensor(0.8277) EO: (tensor(0.0096), tensor(0.0092))
Task 8   Acc: tensor(0.9064) EO: (tensor(0.0264), tensor(0.0167))
Task 9   Acc: tensor(0.7832) EO: (tensor(0.0443), tensor(0.0345))
Task 10   Acc: tensor(0.8559) EO: (tensor(0.0179), tensor(0.0173))
Task 11   Acc: tensor(0.9312) EO: (tensor(0.0148), tensor(0.0128))
Task 12   Acc: tensor(0.9275) EO: (tensor(0.0439), tensor(0.0418))
Task 

Task 28   Acc: tensor(0.8111) EO: (tensor(0.0083), tensor(0.0027))
Task 29   Acc: tensor(0.8790) EO: (tensor(0.0330), tensor(0.0311))
Task 30   Acc: tensor(0.9307) EO: (tensor(0.0210), tensor(0.0209))
Task 0   Acc: tensor(0.8833) EO: (tensor(0.0134), tensor(0.0039))
Task 1   Acc: tensor(0.7658) EO: (tensor(0.0165), tensor(0.0131))
Task 2   Acc: tensor(0.7006) EO: (tensor(0.0488), tensor(0.0240))
Task 3   Acc: tensor(0.7929) EO: (tensor(0.0051), tensor(0.0030))
Task 4   Acc: tensor(0.8720) EO: (tensor(0.0030), tensor(0.0026))
Task 5   Acc: tensor(0.8475) EO: (tensor(0.0370), tensor(0.0326))
Task 6   Acc: tensor(0.7691) EO: (tensor(0.0308), tensor(0.0213))
Task 7   Acc: tensor(0.8264) EO: (tensor(0.0156), tensor(0.0131))
Task 8   Acc: tensor(0.9098) EO: (tensor(0.0580), tensor(0.0499))
Task 9   Acc: tensor(0.7828) EO: (tensor(0.0210), tensor(0.0120))
Task 10   Acc: tensor(0.8606) EO: (tensor(0.0278), tensor(0.0164))
Task 11   Acc: tensor(0.9296) EO: (tensor(0.0216), tensor(0.0134))
Task 

Task 27   Acc: tensor(0.8101) EO: (tensor(0.0554), tensor(0.0523))
Task 28   Acc: tensor(0.7995) EO: (tensor(0.0278), tensor(0.0221))
Task 29   Acc: tensor(0.8790) EO: (tensor(0.0330), tensor(0.0311))
Task 30   Acc: tensor(0.9242) EO: (tensor(0.0305), tensor(0.0264))
Task 0   Acc: tensor(0.8828) EO: (tensor(0.0163), tensor(0.0140))
Task 1   Acc: tensor(0.7549) EO: (tensor(0.0220), tensor(0.0193))
Task 2   Acc: tensor(0.6988) EO: (tensor(0.0263), tensor(0.0078))
Task 3   Acc: tensor(0.7852) EO: (tensor(0.0199), tensor(0.0038))
Task 4   Acc: tensor(0.8738) EO: (tensor(0.0042), tensor(0.0024))
Task 5   Acc: tensor(0.8387) EO: (tensor(0.0258), tensor(0.0242))
Task 6   Acc: tensor(0.7613) EO: (tensor(0.0224), tensor(0.0195))
Task 7   Acc: tensor(0.8177) EO: (tensor(0.0140), tensor(0.0076))
Task 8   Acc: tensor(0.9090) EO: (tensor(0.0152), tensor(0.0054))
Task 9   Acc: tensor(0.7752) EO: (tensor(0.0081), tensor(0.0072))
Task 10   Acc: tensor(0.8568) EO: (tensor(0.0184), tensor(0.0170))
Task 

...

Task 23   Acc: tensor(0.9255) EO: (tensor(0.0214), tensor(0.0172))
Task 24   Acc: tensor(0.6669) EO: (tensor(0.0202), tensor(0.0098))
Task 25   Acc: tensor(0.7889) EO: (tensor(0.0241), tensor(0.0126))
Task 26   Acc: tensor(0.7753) EO: (tensor(0.0020), tensor(0.0011))
Task 27   Acc: tensor(0.7984) EO: (tensor(0.0482), tensor(0.0420))
Task 28   Acc: tensor(0.7965) EO: (tensor(0.0138), tensor(0.0082))
Task 29   Acc: tensor(0.8748) EO: (tensor(0.0363), tensor(0.0345))
Task 30   Acc: tensor(0.9265) EO: (tensor(0.0228), tensor(0.0173))
Task 0   Acc: tensor(0.8673) EO: (tensor(0.0534), tensor(0.0429))
Task 1   Acc: tensor(0.7432) EO: (tensor(0.0109), tensor(0.0020))
Task 2   Acc: tensor(0.6886) EO: (tensor(0.0272), tensor(0.0057))
Task 3   Acc: tensor(0.7633) EO: (tensor(0.0074), tensor(0.0026))
Task 4   Acc: tensor(0.8646) EO: (tensor(0.0108), tensor(0.0038))
Task 5   Acc: tensor(0.8011) EO: (tensor(0.0419), tensor(0.0411))
Task 6   Acc: tensor(0.7415) EO: (tensor(0.0217), tensor(0.0073))
Ta

Task 22   Acc: tensor(0.9095) EO: (tensor(0.0210), tensor(0.0163))
Task 23   Acc: tensor(0.9184) EO: (tensor(0.0375), tensor(0.0368))
Task 24   Acc: tensor(0.6396) EO: (tensor(0.0140), tensor(0.0096))
Task 25   Acc: tensor(0.7572) EO: (tensor(0.0064), tensor(0.0041))
Task 26   Acc: tensor(0.7435) EO: (tensor(0.0354), tensor(0.0332))
Task 27   Acc: tensor(0.7745) EO: (tensor(0.0476), tensor(0.0429))
Task 28   Acc: tensor(0.7771) EO: (tensor(0.0123), tensor(0.0104))
Task 29   Acc: tensor(0.8421) EO: (tensor(0.0172), tensor(0.0064))
Task 30   Acc: tensor(0.9191) EO: (tensor(0.0125), tensor(0.0023))
Task 0   Acc: tensor(0.8535) EO: (tensor(0.0050), tensor(0.0013))
Task 1   Acc: tensor(0.7315) EO: (tensor(0.0321), tensor(0.0243))
Task 2   Acc: tensor(0.6733) EO: (tensor(0.0460), tensor(0.0347))
Task 3   Acc: tensor(0.7425) EO: (tensor(0.0139), tensor(0.0066))
Task 4   Acc: tensor(0.8628) EO: (tensor(0.0499), tensor(0.0398))
Task 5   Acc: tensor(0.7507) EO: (tensor(0.0223), tensor(0.0180))
T

Task 21   Acc: tensor(0.6934) EO: (tensor(0.0164), tensor(0.0101))
Task 22   Acc: tensor(0.9156) EO: (tensor(0.0226), tensor(0.0186))
Task 23   Acc: tensor(0.9201) EO: (tensor(0.0257), tensor(0.0210))
Task 24   Acc: tensor(0.6634) EO: (tensor(0.0015), tensor(0.0002))
Task 25   Acc: tensor(0.7744) EO: (tensor(0.0211), tensor(0.0160))
Task 26   Acc: tensor(0.7616) EO: (tensor(0.0202), tensor(0.0115))
Task 27   Acc: tensor(0.7932) EO: (tensor(0.0593), tensor(0.0570))
Task 28   Acc: tensor(0.7949) EO: (tensor(0.0212), tensor(0.0081))
Task 29   Acc: tensor(0.8533) EO: (tensor(0.0526), tensor(0.0520))
Task 30   Acc: tensor(0.9193) EO: (tensor(0.0207), tensor(0.0195))
Task 0   Acc: tensor(0.8738) EO: (tensor(0.0323), tensor(0.0197))
Task 1   Acc: tensor(0.7355) EO: (tensor(0.0189), tensor(0.0148))
Task 2   Acc: tensor(0.6881) EO: (tensor(0.0357), tensor(0.0201))
Task 3   Acc: tensor(0.7656) EO: (tensor(0.0104), tensor(0.0094))
Task 4   Acc: tensor(0.8625) EO: (tensor(0.0100), tensor(0.0013))


Task 20   Acc: tensor(0.6695) EO: (tensor(0.0119), tensor(0.0024))
Task 21   Acc: tensor(0.6786) EO: (tensor(0.0030), tensor(0.0002))
Task 22   Acc: tensor(0.9061) EO: (tensor(0.0119), tensor(0.0030))
Task 23   Acc: tensor(0.9173) EO: (tensor(0.0291), tensor(0.0273))
Task 24   Acc: tensor(0.6451) EO: (tensor(0.0032), tensor(0.0023))
Task 25   Acc: tensor(0.7617) EO: (tensor(0.0302), tensor(0.0247))
Task 26   Acc: tensor(0.7463) EO: (tensor(0.0142), tensor(0.0096))
Task 27   Acc: tensor(0.7820) EO: (tensor(0.0577), tensor(0.0504))
Task 28   Acc: tensor(0.7841) EO: (tensor(0.0447), tensor(0.0383))
Task 29   Acc: tensor(0.8311) EO: (tensor(0.0479), tensor(0.0459))
Task 30   Acc: tensor(0.9115) EO: (tensor(0.0223), tensor(0.0206))
Task 0   Acc: tensor(0.8559) EO: (tensor(0.0071), tensor(0.0031))
Task 1   Acc: tensor(0.7275) EO: (tensor(0.0275), tensor(0.0081))
Task 2   Acc: tensor(0.6563) EO: (tensor(0.0404), tensor(0.0215))
Task 3   Acc: tensor(0.7402) EO: (tensor(0.0227), tensor(0.0099))

Task 19   Acc: tensor(0.8074) EO: (tensor(0.0035), tensor(0.0020))
Task 20   Acc: tensor(0.6426) EO: (tensor(0.0219), tensor(0.0118))
Task 21   Acc: tensor(0.6538) EO: (tensor(0.0334), tensor(0.0306))
Task 22   Acc: tensor(0.8970) EO: (tensor(0.0129), tensor(0.0033))
Task 23   Acc: tensor(0.9184) EO: (tensor(0.0326), tensor(0.0273))
Task 24   Acc: tensor(0.6390) EO: (tensor(0.0508), tensor(0.0158))
Task 25   Acc: tensor(0.7474) EO: (tensor(0.0417), tensor(0.0311))
Task 26   Acc: tensor(0.7349) EO: (tensor(0.0159), tensor(0.0155))
Task 27   Acc: tensor(0.7699) EO: (tensor(0.0432), tensor(0.0358))
Task 28   Acc: tensor(0.7747) EO: (tensor(0.0190), tensor(0.0122))
Task 29   Acc: tensor(0.8306) EO: (tensor(0.0611), tensor(0.0558))
Task 30   Acc: tensor(0.9152) EO: (tensor(0.0156), tensor(0.0129))
Task 0   Acc: tensor(0.8513) EO: (tensor(0.0299), tensor(0.0179))
Task 1   Acc: tensor(0.7215) EO: (tensor(0.0065), tensor(0.0064))
Task 2   Acc: tensor(0.6708) EO: (tensor(0.0292), tensor(0.0235)

In [7]:
test_batch=next(iter(dataloader_test))[0]

In [9]:
shapes=[1024,128,256,512,128,64,256,128]
depth=len(shapes)-1
spaths={'path'+str(t):'/yourpath/Grouping_FMTL/models/stl/age_stl'+str(t)+'.pt' for t in range(N_tasks)}

In [10]:
snet=nn.DataParallel(STL_vission(pretrain=model_pre.to(dv),shapes=shapes))
snet.to(dv)
Bests=[]
for t in range(N_tasks):  
    snet.load_state_dict(torch.load(spaths['path'+str(t)]))
    pred0=snet(test_batch.to(dv).float())
    accuracy=acc(pred0.to(cpu),torch.tensor(y_test[t]).to(cpu))
    DM=DM_rate(pred0.to(cpu),torch.tensor(y_test[t]).to(cpu),torch.tensor(g_test).to(cpu))
    Bests.append([accuracy,DM])
    print('Task',t,'  Acc:',accuracy, 'DMs:',DM)

Task 0   Acc: tensor(0.8661) DMs: (tensor(0.0431), tensor(0.0408))
Task 1   Acc: tensor(0.7240) DMs: (tensor(0.0114), tensor(0.0097))
Task 2   Acc: tensor(0.6777) DMs: (tensor(0.0166), tensor(0.0064))
Task 3   Acc: tensor(0.7454) DMs: (tensor(0.0062), tensor(0.0050))
Task 4   Acc: tensor(0.8500) DMs: (tensor(0.0200), tensor(0.0158))
Task 5   Acc: tensor(0.6408) DMs: (tensor(0.0097), tensor(0.0003))
Task 6   Acc: tensor(0.7424) DMs: (tensor(0.0108), tensor(0.0034))
Task 7   Acc: tensor(0.7763) DMs: (tensor(0.0197), tensor(0.0168))
Task 8   Acc: tensor(0.9034) DMs: (tensor(0.0156), tensor(0.0149))
Task 9   Acc: tensor(0.7831) DMs: (tensor(0.0143), tensor(0.0134))
Task 10   Acc: tensor(0.8255) DMs: (tensor(0.0122), tensor(0.0096))
Task 11   Acc: tensor(0.9291) DMs: (tensor(0.0137), tensor(0.0124))
Task 12   Acc: tensor(0.9308) DMs: (tensor(0.0133), tensor(0.0097))
Task 13   Acc: tensor(0.7542) DMs: (tensor(0.0166), tensor(0.0134))
Task 14   Acc: tensor(0.6371) DMs: (tensor(0.0125), tensor

In [ ]:
torch.save(torch.tensor(list(np.array(Bests)[:,0])),'/yourpath/Grouping_FMTL/age_sacc.pt')

In [ ]:
torch.save(torch.tensor(list(np.array(Bests)[:,1])),'/yourpath/Grouping_FMTL/age_sf.pt')